In [ ]:
#import packages
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pymannkendall as mk
import cartopy.crs as ccrs
import time
import matplotlib as mpl
from scipy.stats import linregress
import dask
import matplotlib.ticker as mticker
import string
letters=[]
for letter in string.ascii_lowercase[0:24]:
    letters.append(letter+". ")

In [ ]:
from functions import *
era_land=xr.open_dataset('eralandmask_regid.nc').rename({'lat':'latitude','lon':'longitude'})

## Read Hot spells

In [ ]:
#models
###'HadGEM3-GC31-MM'
models1=['ACCESS-CM2','AWI-CM-1-1-MR','BCC-CSM2-MR', 'CanESM5','CESM2','CMCC-ESM2','CNRM-ESM2-1','EC-Earth3-CC','GFDL-ESM4','IITM-ESM','INM-CM5-0','KIOST-ESM','MIROC6','MPI-ESM1-2-LR','MRI-ESM2-0','NESM3','NorESM2-MM','TaiESM1']# temp_datcut={}
models22=['CESM2-WACCM', 'CMCC-CM2-SR5', 'CNRM-CM6-1','INM-CM4-8','IPSL-CM6A-LR', 'MIROC-ES2L','CNRM-CM6-1-HR']
models=models1+models22#['ACCESS-CM2','AWI-CM-1-1-MR','BCC-CSM2-MR', 'CanESM5','CESM2', 'CESM2-WACCM','CMCC-ESM2', 'CMCC-CM2-SR5','CNRM-ESM2-1', 'CNRM-CM6-1', 'CNRM-CM6-1-HR','EC-Earth3-CC','GFDL-ESM4','IITM-ESM','INM-CM4-8', 'INM-CM5-0', 'IPSL-CM6A-LR','KIOST-ESM','MIROC6', 'MIROC-ES2L','MPI-ESM1-2-LR','MRI-ESM2-0','NESM3','NorESM2-MM','TaiESM1']
len(models)

In [ ]:
#get hw data
import dask
heat_dat={}
toconc=[]
for i,mod in enumerate(models):
    try:
        heat=xr.open_dataset('cmip6/heat/'+mod+'_tmean_heatdays_hist_ssp5_1950_2024_regridcon.nc').rename({'lat':'latitude','lon':'longitude'})
    except:
        heat=xr.open_dataset('cmip6/heat/'+mod+'_tmean_heatdays_hist_ssp5_1950_2024_regridcon.nc')
    heat1=heat.sortby(heat.time)
    heat_midl=cutmidlat(heat1)
    heat_midlat=landmask(heat_midl)
    heat_midlat=heat_midlat.drop_duplicates(dim="time")
    if i >0:
        heat_midlat['time']=heat_dat['ACCESS-CM2'].time.values
    else:
        pass
    heat_dat[mod]=heat_midlat
    toconc.append(heat_midlat)

In [ ]:
#concatenate models
heat_mult=xr.concat(toconc,dim='models')
heat_mult.to_netcdf('cmip6/heat/cmip6_hot_25mods_1950-2021.nc')


In [ ]:
heat_mult=xr.open_dataset('cmip6/heat/cmip6_hot_25mods_1950-2021.nc')
heat_mult21=heat_mult.sel(time=slice('1950-01-01','2021-12-31'))

In [ ]:
#read ERA5 and berkley

heat=xr.open_dataset('era5/era5_tmean_heatdays_1950_2024_regrid.nc')
heat1=heat.sortby(heat.time)
#heat79=cuttimestart(heat1,1979)
heat_midl=cutmidlat(heat1)


heat_midlat=landmask(heat_midl)
heat_midlat21=heat_midlat.sel(time=slice('1950-01-01','2021-12-31'))


heat_bk=xr.open_dataset('berkley_tmean_heatdays_1950_2021_regrid.nc')
heat1_bk=heat_bk.sortby(heat_bk.time)
#heat79=cuttimestart(heat1,1979)
heatbk_midl=cutmidlat(heat1_bk)

heatbk_midlat=landmask(heatbk_midl)

In [ ]:
#seasonal average
heat_seas=heat_mult21.groupby('time.season').mean(dim=['time','models'])
# ssns=[]
# heats=[]
# for i in range(4):
#     heats.append(heat_seas.htdays.isel(season=i).stack(z={'latitude','longitude'}).dropna(dim='z').values)
#     ssns.append(str(heat_seas.htdays.isel(season=i).season.values))
    
#heatera_seas=heat_midlat21.groupby('time.season').sum()
##prepare data
# heat_seas=heat_midlat21.groupby('time.year')
# titles=heatera_seas2.season.values
# year= np.arange(1950,2022)
# heat_dy_seas=xr.full_like(heat_midlat21.t2m.groupby('time.season').mean().expand_dims(year=year).copy(),np.nan)

# trseas_mn=[]
# pvseas_mn=[]
# for i,ss in enumerate(year):
#     heatdy=heat_seas[ss].t2m.groupby('time.season').sum()
#     heat_dy_seas[i,]=heatdy
# heatera_seas1=heat_dy_seas.mean(dim='year')#assign_coords(models=heat_dy_seas.models).to_dataset(name='htdays')

heatera_seas=heat_midlat21.groupby('time.season').mean(dim=['time'])
# heateras=[]
# for i in range(4):
#     heateras.append(heatera_seas.t2m.isel(season=i).stack(z={'latitude','longitude'}).dropna(dim='z').values)

heatbk_seas=heatbk_midlat.groupby('time.season').mean(dim=['time'])
# heatbks=[]
# for i in range(4):
#     heatbks.append(heatbk_seas.t2m.isel(season=i).stack(z={'latitude','longitude'}).dropna(dim='z').values)


## Read detrended hotspells

In [ ]:
#models
###'HadGEM3-GC31-MM'
models1=['ACCESS-CM2','AWI-CM-1-1-MR','BCC-CSM2-MR', 'CanESM5','CESM2','CMCC-ESM2','CNRM-ESM2-1','EC-Earth3-CC','GFDL-ESM4','IITM-ESM','INM-CM5-0','KIOST-ESM','MIROC6','MPI-ESM1-2-LR','MRI-ESM2-0','NESM3','NorESM2-MM','TaiESM1']# temp_datcut={}
models22=['CESM2-WACCM', 'CMCC-CM2-SR5', 'CNRM-CM6-1','INM-CM4-8','IPSL-CM6A-LR', 'MIROC-ES2L','CNRM-CM6-1-HR']
models=models1+models22#['ACCESS-CM2','AWI-CM-1-1-MR','BCC-CSM2-MR', 'CanESM5','CESM2', 'CESM2-WACCM','CMCC-ESM2', 'CMCC-CM2-SR5','CNRM-ESM2-1', 'CNRM-CM6-1', 'CNRM-CM6-1-HR','EC-Earth3-CC','GFDL-ESM4','IITM-ESM','INM-CM4-8', 'INM-CM5-0', 'IPSL-CM6A-LR','KIOST-ESM','MIROC6', 'MIROC-ES2L','MPI-ESM1-2-LR','MRI-ESM2-0','NESM3','NorESM2-MM','TaiESM1']
len(models)

In [ ]:
# #get hw data
import dask
detheat_dat={}
dettoconc=[]
for i,mod in enumerate(models):
    heat=xr.open_dataset('cmip6/heat/'+mod+'_detrendedtmean_heatdays_hist_ssp5_1950_2024_regridcon.nc')#.rename({'lat':'latitude','lon':'longitude'})
    heat1=heat.sortby(heat.time)
    heat_midl=cutmidlat(heat1)
    heat_midlat=landmask(heat_midl)
    heat_midlat=heat_midlat.drop_duplicates(dim="time")
    if i >0:
        heat_midlat['time']=detheat_dat['ACCESS-CM2'].time.values
    else:
        pass
    try:
        heat_midlat=heat_midlat.drop_vars('height')
        heat_midlat=heat_midlat.drop_vars('dayofyear')
        heat_midlat=heat_midlat.drop_vars('models')
    except:
        pass
    detheat_dat[mod]=heat_midlat
    dettoconc.append(heat_midlat.htdays)

In [ ]:
#concatenate models
detheat_mult=xr.concat(dettoconc,dim='models')
detheat_mult.to_netcdf('cmip6/heat/cmip6_dethot_25mods_1950-2021.nc')


In [ ]:
detheat_mult=xr.open_dataset('cmip6/heat/cmip6_dethot_25mods_1950-2021.nc')
detheat_mult21=detheat_mult.sel(time=slice('1950-01-01','2021-12-31'))

In [ ]:
#read ERA5 and berkley

detheat=xr.open_dataset('era5/era5_detrendedtmean_heatdays_1950_2024_regrid.nc')
detheat1=detheat.sortby(detheat.time)
#heat79=cuttimestart(heat1,1979)
detheat_midl=cutmidlat(detheat1)


detheat_midlat=landmask(detheat_midl)
detheat_midlat21=detheat_midlat.sel(time=slice('1950-01-01','2021-12-31'))


detheat_bk=xr.open_dataset('berkley_detrendedtmean_heatdays_1950_2021_regrid.nc')
detheat1_bk=detheat_bk.sortby(detheat_bk.time)
#heat79=cuttimestart(heat1,1979)
detheatbk_midl=cutmidlat(detheat1_bk)

detheatbk_midlat=landmask(detheatbk_midl)

In [ ]:
#seasonal average
#detheat_mult21=detheat_mult21.to_dataset(name='htdays')
detheat_seas_m=detheat_mult21.htdays.groupby('time.season').mean(dim=['time']).mean(dim=['models'])
count_days=[90,92,92,91]
detheat_seas=detheat_seas_m.copy()
for i in range(4):
    detheat_seas[i,]=count_days[i]*detheat_seas_m[i,]

# ssns=[]
# detheats=[]
# for i in range(4):
#     detheats.append(detheat_seas.htdays.isel(season=i).stack(z={'latitude','longitude'}).dropna(dim='z').values)
#     ssns.append(str(detheat_seas.htdays.isel(season=i).season.values))
    
detheatera_seas_m=detheat_midlat21.groupby('time.season').mean(dim=['time'])
count_days=[90,92,92,91]
detheatera_seas=detheatera_seas_m.t2m.copy()
for i in range(4):
    detheatera_seas[i,]=count_days[i]*detheatera_seas_m.t2m[i,]
# detheateras=[]
# for i in range(4):
#     detheateras.append(detheatera_seas.t2m.isel(season=i).stack(z={'latitude','longitude'}).dropna(dim='z').values)

detheatbk_seas_m=detheatbk_midlat.groupby('time.season').mean(dim=['time'])
count_days=[90,92,92,91]
detheatbk_seas=detheatbk_seas_m.t2m.copy()
for i in range(4):
    detheatbk_seas[i,]=count_days[i]*detheatbk_seas_m.t2m[i,]
# detheatbks=[]
# for i in range(4):
#     detheatbks.append(detheatbk_seas.t2m.isel(season=i).stack(z={'latitude','longitude'}).dropna(dim='z').values)


## seasonal hotspell trends

### CMIP6

In [ ]:
#prepare data
heat_seas=heat_mult21.groupby('time.season')
titles=heat_seas.mean().season.values
# year= np.arange(1950,2025)
heat_dy_seas=xr.full_like(heat_mult21.htdays.groupby('time.year').mean().expand_dims(season=titles).copy(),np.nan)

trseas_mn=[]
pvseas_mn=[]
for i,ss in enumerate(titles):
    heatdy=heat_seas[ss].htdays.groupby('time.year').mean()
    heat_dy_seas[i,]=heatdy
heat_dy_seas=heat_dy_seas.assign_coords(models=heat_dy_seas.models).to_dataset(name='htdays')

In [ ]:
#calc slope
from dask.distributed import Client
client = Client(n_workers=20, threads_per_worker=1,  memory_limit='12GB') 
heatdy_seasch=heat_dy_seas.chunk({'latitude': 13, 'longitude': 12,'year': -1})
slope, pval = xr.apply_ufunc(
    manntrend,
    heatdy_seasch.htdays,  # your variable (time on first axis)
    input_core_dims=[['year']],
    output_core_dims=[[], []],
    vectorize=True,
    dask='parallelized',
    output_dtypes=[float, float]
)

#compute dask
trseas_mn,pvseas_mn=dask.compute(slope,pval)

In [ ]:
#save
trseas_mn.to_dataset(name='htdays').to_netcdf('cmip6/heat/cmip6_hot_trend_25mods_1950-2021.nc')
pvseas_mn.to_dataset(name='htdays').to_netcdf('cmip6/heat/cmip6_hot_trendpvals_25mods_1950-2021.nc')


### ERA5

In [ ]:
#era5 trend
##mann kendall version
heat_ch=heat_midlat21.chunk({'latitude': 26, 'longitude': 60})
heat_seas=heat_ch.groupby('time.season')

titles=heat_seas.mean().season.values
# year= np.arange(1950,2025)
heat_dy_seas=xr.full_like(heat_midlat21.t2m.groupby('time.year').mean().expand_dims(season=titles).copy(),np.nan)

for i,ss in enumerate(titles):
    heatdy=heat_seas[ss].t2m.groupby('time.year').mean()
    heat_dy_seas[i,]=heatdy
heat_dy_seas=heat_dy_seas.to_dataset(name='htdays')

heatdy_seasch=heat_dy_seas.chunk({'latitude': 13, 'longitude': 12,'year': -1})
slope, pval = xr.apply_ufunc(
    manntrend,
    heatdy_seasch.htdays,  # your variable (time on first axis)
    input_core_dims=[['year']],
    output_core_dims=[[], []],
    vectorize=True,
    dask='parallelized',
    output_dtypes=[float, float]
)

trseas_mn_era,pvseas_mn_era=dask.compute(slope,pval)

In [ ]:
#save
trseas_mn_era.to_dataset(name='htdays').to_netcdf('era5/era5_hot_trend_final_1950-2021.nc')
pvseas_mn_era.to_dataset(name='htdays').to_netcdf('era5/era5_hot_trendpvals_final_1950-2021.nc')


### Berkeley

In [ ]:
#berkley trend
##mann kendall version
heat_ch=heatbk_midlat.chunk({'latitude': 26, 'longitude': 60})
heat_seas=heat_ch.groupby('time.season')

titles=heat_seas.mean().season.values
# year= np.arange(1950,2025)
heat_dy_seas=xr.full_like(heatbk_midlat.t2m.groupby('time.year').mean().expand_dims(season=titles).copy(),np.nan)

for i,ss in enumerate(titles):
    heatdy=heat_seas[ss].t2m.groupby('time.year').mean()
    heat_dy_seas[i,]=heatdy
heat_dy_seas=heat_dy_seas.to_dataset(name='htdays')

heatdy_seasch=heat_dy_seas.chunk({'latitude': 13, 'longitude': 12,'year': -1})
slope, pval = xr.apply_ufunc(
    manntrend,
    heatdy_seasch.htdays,  # your variable (time on first axis)
    input_core_dims=[['year']],
    output_core_dims=[[], []],
    vectorize=True,
    dask='parallelized',
    output_dtypes=[float, float]
)

trseas_mn_bk,pvseas_mn_bk=dask.compute(slope,pval)

In [ ]:
#save
trseas_mn_bk.to_dataset(name='htdays').to_netcdf('berkeley/berkeley_hot_trend_final_1950-2021.nc')
pvseas_mn_bk.to_dataset(name='htdays').to_netcdf('berkeley/berkeley_hot_trendpvals_final_1950-2021.nc')


## SH hotspell trends

### CMIP6

In [ ]:
#read get hw data
import dask
heat_dat={}
toconc=[]
for i,mod in enumerate(models):
    try:
        heat=xr.open_dataset('cmip6/heat/'+mod+'_tmean_heatdays_hist_ssp5_1950_2024_regridcon.nc').rename({'lat':'latitude','lon':'longitude'})
    except:
        heat=xr.open_dataset('cmip6/heat/'+mod+'_tmean_heatdays_hist_ssp5_1950_2024_regridcon.nc')
    heat1=heat.sortby(heat.time)
    heat_midl=heat1.sel(latitude=slice(-90,-23))
    heat_midlat=landmask(heat_midl)
    heat_midlat=heat_midlat.drop_duplicates(dim="time")
    if i >0:
        heat_midlat['time']=heat_dat['ACCESS-CM2'].time.values
    else:
        pass
    heat_dat[mod]=heat_midlat
    toconc.append(heat_midlat)

In [ ]:
#concatenate models
heat_mult=xr.concat(toconc,dim='models')
heat_mult21=heat_mult.sel(time=slice('1950-01-01','2021-12-31'))

In [ ]:
#prepare data
heat_seas=heat_mult21.groupby('time.season')
titles=heat_seas.mean().season.values
# year= np.arange(1950,2025)
heat_dy_seas=xr.full_like(heat_mult21.htdays.groupby('time.year').mean().expand_dims(season=titles).copy(),np.nan)

trseas_mn=[]
pvseas_mn=[]
for i,ss in enumerate(titles):
    heatdy=heat_seas[ss].htdays.groupby('time.year').mean()
    heat_dy_seas[i,]=heatdy
heat_dy_seas=heat_dy_seas.assign_coords(models=heat_dy_seas.models).to_dataset(name='htdays')

In [ ]:
#calc slope
from dask.distributed import Client
client = Client(n_workers=20, threads_per_worker=1,  memory_limit='12GB') 
heatdy_seasch=heat_dy_seas.chunk({'latitude': 13, 'longitude': 12,'year': -1})
slope, pval = xr.apply_ufunc(
    manntrend,
    heatdy_seasch.htdays,  # your variable (time on first axis)
    input_core_dims=[['year']],
    output_core_dims=[[], []],
    vectorize=True,
    dask='parallelized',
    output_dtypes=[float, float]
)

#compute dask
trseas_mn,pvseas_mn=dask.compute(slope,pval)

In [ ]:
#save
trseas_mn.to_dataset(name='htdays').to_netcdf('cmip6/heat/cmip6_SH_hot_trend_25mods_1950-2021.nc')
pvseas_mn.to_dataset(name='htdays').to_netcdf('cmip6/heat/cmip6_SH_hot_trendpvals_25mods_1950-2021.nc')


### ERA5

In [ ]:
#read ERA5 

heat=xr.open_dataset('era5/era5_tmean_heatdays_1950_2024_regrid.nc')
heat1=heat.sortby(heat.time)
#heat79=cuttimestart(heat1,1979)
heat_midl=heat1.sel(latitude=slice(-90,-23))


heat_midlat=landmask(heat_midl)
heat_midlat21=heat_midlat.sel(time=slice('1950-01-01','2021-12-31'))


heat_bk=xr.open_dataset('berkley_tmean_heatdays_1950_2021_regrid.nc')
heat1_bk=heat_bk.sortby(heat_bk.time)
#heat79=cuttimestart(heat1,1979)
heatbk_midl=heat1_bk.sel(latitude=slice(-90,-23))

heatbk_midlat=landmask(heatbk_midl)

In [ ]:
#era5 trend
##mann kendall version
heat_ch=heat_midlat21.chunk({'latitude': 26, 'longitude': 60})
heat_seas=heat_ch.groupby('time.season')

titles=heat_seas.mean().season.values
# year= np.arange(1950,2025)
heat_dy_seas=xr.full_like(heat_midlat21.t2m.groupby('time.year').mean().expand_dims(season=titles).copy(),np.nan)

for i,ss in enumerate(titles):
    heatdy=heat_seas[ss].t2m.groupby('time.year').mean()
    heat_dy_seas[i,]=heatdy
heat_dy_seas=heat_dy_seas.to_dataset(name='htdays')

heatdy_seasch=heat_dy_seas.chunk({'latitude': 13, 'longitude': 12,'year': -1})
slope, pval = xr.apply_ufunc(
    manntrend,
    heatdy_seasch.htdays,  # your variable (time on first axis)
    input_core_dims=[['year']],
    output_core_dims=[[], []],
    vectorize=True,
    dask='parallelized',
    output_dtypes=[float, float]
)

trseas_mn_era,pvseas_mn_era=dask.compute(slope,pval)

In [ ]:
#save
trseas_mn_era.to_dataset(name='htdays').to_netcdf('era5/era5_SH_hot_trend_final_1950-2021.nc')
pvseas_mn_era.to_dataset(name='htdays').to_netcdf('era5/era5_SH_hot_trendpvals_final_1950-2021.nc')


### Berkeley

In [ ]:
#berkley trend
##mann kendall version
heat_ch=heatbk_midlat.chunk({'latitude': 26, 'longitude': 60})
heat_seas=heat_ch.groupby('time.season')

titles=heat_seas.mean().season.values
# year= np.arange(1950,2025)
heat_dy_seas=xr.full_like(heatbk_midlat.t2m.groupby('time.year').mean().expand_dims(season=titles).copy(),np.nan)

for i,ss in enumerate(titles):
    heatdy=heat_seas[ss].t2m.groupby('time.year').mean()
    heat_dy_seas[i,]=heatdy
heat_dy_seas=heat_dy_seas.to_dataset(name='htdays')

heatdy_seasch=heat_dy_seas.chunk({'latitude': 13, 'longitude': 12,'year': -1})
slope, pval = xr.apply_ufunc(
    manntrend,
    heatdy_seasch.htdays,  # your variable (time on first axis)
    input_core_dims=[['year']],
    output_core_dims=[[], []],
    vectorize=True,
    dask='parallelized',
    output_dtypes=[float, float]
)

trseas_mn_bk,pvseas_mn_bk=dask.compute(slope,pval)

In [ ]:
#save
trseas_mn_bk.to_dataset(name='htdays').to_netcdf('berkeley/berkeley_SH_hot_trend_final_1950-2021.nc')
pvseas_mn_bk.to_dataset(name='htdays').to_netcdf('berkeley/berkeley_SH_hot_trendpvals_final_1950-2021.nc')


## Temperature trend

### CMIP6

In [ ]:
#models
models1=['ACCESS-CM2','AWI-CM-1-1-MR','BCC-CSM2-MR', 'CanESM5','CESM2','CMCC-ESM2','CNRM-ESM2-1','EC-Earth3-CC','GFDL-ESM4','IITM-ESM','INM-CM5-0','KIOST-ESM','MIROC6','MPI-ESM1-2-LR','MRI-ESM2-0','NESM3','NorESM2-MM','TaiESM1']# temp_datcut={}
models22=['CESM2-WACCM', 'CMCC-CM2-SR5', 'CNRM-CM6-1','INM-CM4-8','IPSL-CM6A-LR', 'MIROC-ES2L','CNRM-CM6-1-HR']
models=models1+models22#['ACCESS-CM2','AWI-CM-1-1-MR','BCC-CSM2-MR', 'CanESM5','CESM2', 'CESM2-WACCM','CMCC-ESM2', 'CMCC-CM2-SR5','CNRM-ESM2-1', 'CNRM-CM6-1', 'CNRM-CM6-1-HR','EC-Earth3-CC','GFDL-ESM4','IITM-ESM','INM-CM4-8', 'INM-CM5-0', 'IPSL-CM6A-LR','KIOST-ESM','MIROC6', 'MIROC-ES2L','MPI-ESM1-2-LR','MRI-ESM2-0','NESM3','NorESM2-MM','TaiESM1']


In [ ]:
#get temp data
to_con=[]
# temp_datcut={}
for mod in models1:
    dat=xr.open_dataset('cmip6/hist_ssp_'+mod+'_histssp_regridcon.nc')#.rename({'lat':'latitude','lon':'longitude'})
    dat1=dat.convert_calendar('noleap').rename({'lon':'longitude','lat':'latitude'})
    temp_dat=rotlon_180(dat1)
    tmp1=temp_dat.sortby(temp_dat.time)
    tmp_midl=cutmidlat(tmp1)
    tmp_midlat=landmask(tmp_midl)
    tmp_midlat=tmp_midlat.drop_duplicates(dim="time")
   # temp_datcut[mod]=temp_dat#.sel(time=(slice('1950-01-01','2024-12-31')))
    to_con.append(tmp_midlat)

models2=['cesm2_waccm', 'cmcc_cm2_sr5', 'cnrm_cm6_1','inm_cm4_8','ipsl_cm6a_lr', 'miroc_es2l','cnrm_cm6_1_hr']
# temp_datcut={}
for mod in models2:
    dat=xr.open_dataset('cmip6/tmean/tmean_'+mod+'_histssp_regridcon.nc')#.rename({'lat':'latitude','lon':'longitude'})
    dat1=dat.convert_calendar('noleap').rename({'lon':'longitude','lat':'latitude'})
    temp_dat=rotlon_180(dat1)
    tmp1=temp_dat.sortby(temp_dat.time)
    tmp_midl=cutmidlat(tmp1)
    tmp_midlat=landmask(tmp_midl)
    tmp_midlat=tmp_midlat.drop_duplicates(dim="time")
   # temp_datcut[mod]=temp_dat#.sel(time=(slice('1950-01-01','2024-12-31')))
    to_con.append(tmp_midlat) 

#temp_multfut=xr.open_dataset('cmip6/future/tas_day_allmodels_future_ssp5_regridcon.nc')

In [ ]:
#concatenate and save
temp_mult=xr.concat(to_con,dim='models')
temp_mult=temp_mult.sortby(temp_mult.time)
temp_mult21=temp_mult.sel(time=slice('1950-01-01','2021-12-31'))
temp_lnd=cutmidlat(temp_mult21)
temp_midlat=landmask(temp_lnd)
temp_midlat.to_netcdf('cmip6_tas_day_hist-ssp_25models_1950-2021_regridcon.nc')

In [ ]:
#read cmip
temp_mult=xr.open_dataset('cmip6_tas_day_hist-ssp_25models_1950-2021_regridcon.nc')#.load()


In [ ]:
##mann kendall slope prep
temp_mult21=temp_mult#.to_dataset(name='tas')
temp_seas=temp_mult21.tas.groupby('time.season')
titles=temp_seas.mean().season.values
# year= np.arange(1950,2025)
temp_dy_seas=xr.full_like(temp_mult21.tas.groupby('time.year').mean().expand_dims(season=titles).copy(),np.nan)

trseas_mn=[]
pvseas_mn=[]
for i,ss in enumerate(titles):
    tempdy=temp_seas[ss].groupby('time.year').mean()
    temp_dy_seas[i,]=tempdy
temp_dy_seas=temp_dy_seas.assign_coords(models=temp_dy_seas.models).to_dataset(name='tas')

In [ ]:
#calc slope and save
from dask.distributed import Client
#client = Client(n_workers=20, threads_per_worker=1,  memory_limit='12GB') 
tempdy_seasch=temp_dy_seas.chunk({'latitude': 13, 'longitude': 12,'year': -1})
slope, pval = xr.apply_ufunc(
    manntrend,
    tempdy_seasch.tas,  # your variable (time on first axis)
    input_core_dims=[['year']],
    output_core_dims=[[], []],
    vectorize=True,
    dask='parallelized',
    output_dtypes=[float, float]
)
trseas_mn,pvseas_mn=dask.compute(slope,pval)
trseas_mn.to_dataset(name='tas').to_netcdf('cmip6_tas_day_hist_mktrends.nc')
pvseas_mn.to_dataset(name='tas').to_netcdf('cmip6_tas_day_hist_mkpvals.nc')

### ERA5/Berkeley

In [ ]:
#read ERA5 

tmn=xr.open_dataset('era5/era5_tmean_daily_1950_2024_regridcon.nc')
tmn1=tmn.sortby(tmn.time).rename({'lat':'latitude','lon':'longitude'})
#heat79=cuttimestart(heat1,1979)
tmn_midl=cutmidlat(rotlon_180(tmn1))

tmn_midlat=landmask(tmn_midl)
tmn_midlat21=tmn_midlat.sel(time=slice('1950-01-01','2021-12-31'))
tmn_noleap21=tmn_midlat21.convert_calendar('noleap')

tmn_bk=xr.open_dataset('berkley_tmean_1950-2021_regridcon.nc')
tmn_bk1=tmn_bk.sortby(tmn_bk.time).rename({'lat':'latitude','lon':'longitude'})
#heat79=cuttimestart(heat1,1979)
tmnbk_midl=cutmidlat(rotlon_180(tmn_bk1))

tmnbk_midlat=landmask(tmnbk_midl)
tmnbk_noleap=tmnbk_midlat.convert_calendar('noleap')
#



In [ ]:
#era5 trend
##mann kendall version
tmn_ch=tmn_midlat.chunk({'latitude': 26, 'longitude': 60})
tmn_seas=tmn_ch.groupby('time.season')

titles=tmn_seas.mean().season.values
# year= np.arange(1950,2025)
tmn_dy_seas=xr.full_like(tmn_midlat.t2m.groupby('time.year').mean().expand_dims(season=titles).copy(),np.nan)

for i,ss in enumerate(titles):
    tmndy=tmn_seas[ss].t2m.groupby('time.year').mean()
    tmn_dy_seas[i,]=tmndy
tmn_dy_seas=tmn_dy_seas.to_dataset(name='htdays')

tmndy_seasch=tmn_dy_seas.chunk({'latitude': 13, 'longitude': 12,'year': -1})
slope, pval = xr.apply_ufunc(
    manntrend,
    tmndy_seasch.htdays,  # your variable (time on first axis)
    input_core_dims=[['year']],
    output_core_dims=[[], []],
    vectorize=True,
    dask='parallelized',
    output_dtypes=[float, float]
)

trseas_mn_era,pvseas_mn_era=dask.compute(slope,pval)


In [ ]:
#save
trseas_mn_era.to_dataset(name='htdays').to_netcdf('era5/era5_tmean_trend_final_1950-2021.nc')
pvseas_mn_era.to_dataset(name='htdays').to_netcdf('era5/era5_tmean_trendpvals_final_1950-2021.nc')


In [ ]:
#berkley
tmnbk_ch=tmnbk_midlat.chunk({'latitude': 26, 'longitude': 60})
tmnbk_seas=tmnbk_ch.groupby('time.season')

titles=tmnbk_seas.mean().season.values
# year= np.arange(1950,2025)
tmnbk_dy_seas=xr.full_like(tmnbk_midlat.temperature.groupby('time.year').mean().expand_dims(season=titles).copy(),np.nan)

for i,ss in enumerate(titles):
    tmnbkdy=tmnbk_seas[ss].temperature.groupby('time.year').mean()
    tmnbk_dy_seas[i,]=tmnbkdy
tmnbk_dy_seas=tmnbk_dy_seas.to_dataset(name='htdays')

tmnbkdy_seasch=tmnbk_dy_seas.chunk({'latitude': 13, 'longitude': 12,'year': -1})
slope, pval = xr.apply_ufunc(
    manntrend,
    tmnbkdy_seasch.htdays,  # your variable (time on first axis)
    input_core_dims=[['year']],
    output_core_dims=[[], []],
    vectorize=True,
    dask='parallelized',
    output_dtypes=[float, float]
)

trseas_mn_bk,pvseas_mn_bk=dask.compute(slope,pval)

In [ ]:
#save
trseas_mn_bk.to_dataset(name='htdays').to_netcdf('berkeley/berkeley_tmean_trend_final_1950-2021.nc')
pvseas_mn_bk.to_dataset(name='htdays').to_netcdf('berkeley/berkeley_tmean_trendpvals_final_1950-2021.nc')

## Median trends

### CMIP6

In [ ]:
##mann kendall version
temp_mult21=temp_mult
temp_seas=temp_mult21.tas.groupby('time.season')
titles=temp_seas.mean().season.values
# year= np.arange(1950,2025)
temp_dy_seas=xr.full_like(temp_mult21.tas.groupby('time.year').mean().expand_dims(season=titles).copy(),np.nan)

trseas_mn=[]
pvseas_mn=[]
for i,ss in enumerate(titles):
    tempdy=temp_seas[ss].groupby('time.year').median()
    temp_dy_seas[i,]=tempdy
temp_dy_seas=temp_dy_seas.assign_coords(models=temp_dy_seas.models).to_dataset(name='tas')

In [ ]:
#calc slope
from dask.distributed import Client
#client = Client(n_workers=20, threads_per_worker=1,  memory_limit='12GB') 
tempdy_seasch=temp_dy_seas.chunk({'latitude': 13, 'longitude': 12,'year': -1})
slope, pval = xr.apply_ufunc(
    manntrend,
    tempdy_seasch.tas,  # your variable (time on first axis)
    input_core_dims=[['year']],
    output_core_dims=[[], []],
    vectorize=True,
    dask='parallelized',
    output_dtypes=[float, float]
)
trseas_md,pvseas_md=dask.compute(slope,pval)

In [ ]:
#save
trseas_md.to_dataset(name='tas').to_netcdf('cmip6_tas_day_median_hist_mktrends.nc')
pvseas_md.to_dataset(name='tas').to_netcdf('cmip6_tas_day_median_hist_mkpvals.nc')

### ERA5/Berkeley

In [ ]:
#read ERA5 and berkley

tmn=xr.open_dataset('era5/era5_tmean_daily_1950_2024_regridcon.nc')
tmn1=tmn.sortby(tmn.time).rename({'lat':'latitude','lon':'longitude'})
#heat79=cuttimestart(heat1,1979)
tmn_midl=cutmidlat(rotlon_180(tmn1))

tmn_midlat=landmask(tmn_midl)
tmn_midlat21=tmn_midlat.sel(time=slice('1950-01-01','2021-12-31'))
tmn_noleap21=tmn_midlat21.convert_calendar('noleap')

tmn_bk=xr.open_dataset('berkley_tmean_1950-2021_regridcon.nc')
tmn_bk1=tmn_bk.sortby(tmn_bk.time).rename({'lat':'latitude','lon':'longitude'})
#heat79=cuttimestart(heat1,1979)
tmnbk_midl=cutmidlat(rotlon_180(tmn_bk1))

tmnbk_midlat=landmask(tmnbk_midl)
tmnbk_noleap=tmnbk_midlat.convert_calendar('noleap')
#



In [ ]:
#era5 trend
##mann kendall version
tmn_ch=tmn_midlat.chunk({'latitude': 26, 'longitude': 60})
tmn_seas=tmn_ch.groupby('time.season')

titles=tmn_seas.mean().season.values
# year= np.arange(1950,2025)
tmn_dy_seas=xr.full_like(tmn_midlat.t2m.groupby('time.year').mean().expand_dims(season=titles).copy(),np.nan)

for i,ss in enumerate(titles):
    tmndy=tmn_seas[ss].t2m.groupby('time.year').median()
    tmn_dy_seas[i,]=tmndy
tmn_dy_seas=tmn_dy_seas.to_dataset(name='htdays')

tmndy_seasch=tmn_dy_seas.chunk({'latitude': 13, 'longitude': 12,'year': -1})
slope, pval = xr.apply_ufunc(
    manntrend,
    tmndy_seasch.htdays,  # your variable (time on first axis)
    input_core_dims=[['year']],
    output_core_dims=[[], []],
    vectorize=True,
    dask='parallelized',
    output_dtypes=[float, float]
)

trseas_md_era,pvseas_md_era=dask.compute(slope,pval)


In [ ]:
#save
trseas_md_era.to_dataset(name='htdays').to_netcdf('era5/era5_tmean_median_trend_final_1950-2021.nc')
pvseas_md_era.to_dataset(name='htdays').to_netcdf('era5/era5_tmean_median_trendpvals_final_1950-2021.nc')


In [ ]:
#berkley
tmnbk_ch=tmnbk_midlat.chunk({'latitude': 26, 'longitude': 60})
tmnbk_seas=tmnbk_ch.groupby('time.season')

titles=tmnbk_seas.mean().season.values
# year= np.arange(1950,2025)
tmnbk_dy_seas=xr.full_like(tmnbk_midlat.temperature.groupby('time.year').mean().expand_dims(season=titles).copy(),np.nan)

for i,ss in enumerate(titles):
    tmnbkdy=tmnbk_seas[ss].temperature.groupby('time.year').median()
    tmnbk_dy_seas[i,]=tmnbkdy
tmnbk_dy_seas=tmnbk_dy_seas.to_dataset(name='htdays')

tmnbkdy_seasch=tmnbk_dy_seas.chunk({'latitude': 13, 'longitude': 12,'year': -1})
slope, pval = xr.apply_ufunc(
    manntrend,
    tmnbkdy_seasch.htdays,  # your variable (time on first axis)
    input_core_dims=[['year']],
    output_core_dims=[[], []],
    vectorize=True,
    dask='parallelized',
    output_dtypes=[float, float]
)

trseas_md_bk,pvseas_md_bk=dask.compute(slope,pval)

In [ ]:
#save
trseas_md_bk.to_dataset(name='htdays').to_netcdf('berkeley/berkeley_tmean_median_trend_final_1950-2021.nc')
pvseas_md_bk.to_dataset(name='htdays').to_netcdf('berkeley/berkeley_tmean_median_trendpvals_final_1950-2021.nc')

## 25th percentile

### CMIP6

In [ ]:
##mann kendall version

temp_seas=temp_mult21.tas.groupby('time.season')
titles=temp_seas.mean().season.values
# year= np.arange(1950,2025)
temp_dy_seas=xr.full_like(temp_mult21.tas.groupby('time.year').mean().expand_dims(season=titles).copy(),np.nan)

trseas_mn=[]
pvseas_mn=[]
for i,ss in enumerate(titles):
    tempdy=temp_seas[ss].groupby('time.year').quantile(0.25)
    temp_dy_seas[i,]=tempdy
temp_dy_seas=temp_dy_seas.assign_coords(models=temp_dy_seas.models).to_dataset(name='tas')

In [ ]:
#calc slope
from dask.distributed import Client
#client = Client(n_workers=20, threads_per_worker=1,  memory_limit='12GB') 
tempdy_seasch=temp_dy_seas.chunk({'latitude': 13, 'longitude': 12,'year': -1})
slope, pval = xr.apply_ufunc(
    manntrend,
    tempdy_seasch.tas,  # your variable (time on first axis)
    input_core_dims=[['year']],
    output_core_dims=[[], []],
    vectorize=True,
    dask='parallelized',
    output_dtypes=[float, float]
)
trseas_q25,pvseas_q25=dask.compute(slope,pval)

In [ ]:
#save
trseas_q25.to_dataset(name='tas').to_netcdf('cmip6_tas_day_q25_hist_mktrends.nc')
pvseas_q25.to_dataset(name='tas').to_netcdf('cmip6_tas_day_q25_hist_mkpvals.nc')

### ERA5/Berkeley

In [ ]:
#read ERA5 and berkley

tmn=xr.open_dataset('era5/era5_tmean_daily_1950_2024_regridcon.nc')
tmn1=tmn.sortby(tmn.time).rename({'lat':'latitude','lon':'longitude'})
#heat79=cuttimestart(heat1,1979)
tmn_midl=cutmidlat(rotlon_180(tmn1))

tmn_midlat=landmask(tmn_midl)
tmn_midlat21=tmn_midlat.sel(time=slice('1950-01-01','2021-12-31'))
tmn_noleap21=tmn_midlat21.convert_calendar('noleap')

tmn_bk=xr.open_dataset('berkley_tmean_1950-2021_regridcon.nc')
tmn_bk1=tmn_bk.sortby(tmn_bk.time).rename({'lat':'latitude','lon':'longitude'})
#heat79=cuttimestart(heat1,1979)
tmnbk_midl=cutmidlat(rotlon_180(tmn_bk1))

tmnbk_midlat=landmask(tmnbk_midl)
tmnbk_noleap=tmnbk_midlat.convert_calendar('noleap')
#



In [ ]:
#era5 trend
##mann kendall version
tmn_ch=tmn_midlat.chunk({'latitude': 26, 'longitude': 60})
tmn_seas=tmn_ch.groupby('time.season')

titles=tmn_seas.mean().season.values
# year= np.arange(1950,2025)
tmn_dy_seas=xr.full_like(tmn_midlat.t2m.groupby('time.year').mean().expand_dims(season=titles).copy(),np.nan)

for i,ss in enumerate(titles):
    tmndy=tmn_seas[ss].t2m.groupby('time.year').quantile(0.25)
    tmn_dy_seas[i,]=tmndy
tmn_dy_seas=tmn_dy_seas.to_dataset(name='htdays')

tmndy_seasch=tmn_dy_seas.chunk({'latitude': 13, 'longitude': 12,'year': -1})
slope, pval = xr.apply_ufunc(
    manntrend,
    tmndy_seasch.htdays,  # your variable (time on first axis)
    input_core_dims=[['year']],
    output_core_dims=[[], []],
    vectorize=True,
    dask='parallelized',
    output_dtypes=[float, float]
)

trseas_q25_era,pvseas_q25_era=dask.compute(slope,pval)


In [ ]:
#berkley
tmnbk_ch=tmnbk_midlat.chunk({'latitude': 26, 'longitude': 60})
tmnbk_seas=tmnbk_ch.groupby('time.season')

titles=tmnbk_seas.mean().season.values
# year= np.arange(1950,2025)
tmnbk_dy_seas=xr.full_like(tmnbk_midlat.temperature.groupby('time.year').mean().expand_dims(season=titles).copy(),np.nan)

for i,ss in enumerate(titles):
    tmnbkdy=tmnbk_seas[ss].temperature.groupby('time.year').quantile(0.25)
    tmnbk_dy_seas[i,]=tmnbkdy
tmnbk_dy_seas=tmnbk_dy_seas.to_dataset(name='htdays')

tmnbkdy_seasch=tmnbk_dy_seas.chunk({'latitude': 13, 'longitude': 12,'year': -1})
slope, pval = xr.apply_ufunc(
    manntrend,
    tmnbkdy_seasch.htdays,  # your variable (time on first axis)
    input_core_dims=[['year']],
    output_core_dims=[[], []],
    vectorize=True,
    dask='parallelized',
    output_dtypes=[float, float]
)

trseas_q25_bk,pvseas_q25_bk=dask.compute(slope,pval)

In [ ]:
#save
trseas_q25_era.to_dataset(name='htdays').to_netcdf('era5/era5_tmean_q25_trend_final_1950-2021.nc')
pvseas_q25_era.to_dataset(name='htdays').to_netcdf('era5/era5_tmean_q25_trendpvals_final_1950-2021.nc')


In [ ]:
#save
trseas_q25_bk.to_dataset(name='htdays').to_netcdf('berkeley/berkeley_tmean_q25_trend_final_1950-2021.nc')
pvseas_q25_bk.to_dataset(name='htdays').to_netcdf('berkeley/berkeley_tmean_q25_trendpvals_final_1950-2021.nc')

## 75th percentile

### CMIP6

In [ ]:
##mann kendall version

temp_seas=temp_mult21.tas.groupby('time.season')
titles=temp_seas.mean().season.values
# year= np.arange(1950,2025)
temp_dy_seas=xr.full_like(temp_mult21.tas.groupby('time.year').mean().expand_dims(season=titles).copy(),np.nan)

trseas_mn=[]
pvseas_mn=[]
for i,ss in enumerate(titles):
    tempdy=temp_seas[ss].groupby('time.year').quantile(0.75)
    temp_dy_seas[i,]=tempdy
temp_dy_seas=temp_dy_seas.assign_coords(models=temp_dy_seas.models).to_dataset(name='tas')

In [ ]:
#calc slope
from dask.distributed import Client
#client = Client(n_workers=20, threads_per_worker=1,  memory_limit='12GB') 
tempdy_seasch=temp_dy_seas.chunk({'latitude': 13, 'longitude': 12,'year': -1})
slope, pval = xr.apply_ufunc(
    manntrend,
    tempdy_seasch.tas,  # your variable (time on first axis)
    input_core_dims=[['year']],
    output_core_dims=[[], []],
    vectorize=True,
    dask='parallelized',
    output_dtypes=[float, float]
)
trseas_q75,pvseas_q75=dask.compute(slope,pval)

In [ ]:
#save
trseas_q75.to_dataset(name='tas').to_netcdf('cmip6_tas_day_q75_hist_mktrends.nc')
pvseas_q75.to_dataset(name='tas').to_netcdf('cmip6_tas_day_q75_hist_mkpvals.nc')

### ERA5/Berkeley

In [ ]:
#read ERA5 and berkley

tmn=xr.open_dataset('era5/era5_tmean_daily_1950_2024_regridcon.nc')
tmn1=tmn.sortby(tmn.time).rename({'lat':'latitude','lon':'longitude'})
#heat79=cuttimestart(heat1,1979)
tmn_midl=cutmidlat(rotlon_180(tmn1))

tmn_midlat=landmask(tmn_midl)
tmn_midlat21=tmn_midlat.sel(time=slice('1950-01-01','2021-12-31'))
tmn_noleap21=tmn_midlat21.convert_calendar('noleap')

tmn_bk=xr.open_dataset('berkley_tmean_1950-2021_regridcon.nc')
tmn_bk1=tmn_bk.sortby(tmn_bk.time).rename({'lat':'latitude','lon':'longitude'})
#heat79=cuttimestart(heat1,1979)
tmnbk_midl=cutmidlat(rotlon_180(tmn_bk1))

tmnbk_midlat=landmask(tmnbk_midl)
tmnbk_noleap=tmnbk_midlat.convert_calendar('noleap')
#



In [ ]:
#era5 trend
##mann kendall version
tmn_ch=tmn_midlat.chunk({'latitude': 26, 'longitude': 60})
tmn_seas=tmn_ch.groupby('time.season')

titles=tmn_seas.mean().season.values
# year= np.arange(1950,2025)
tmn_dy_seas=xr.full_like(tmn_midlat.t2m.groupby('time.year').mean().expand_dims(season=titles).copy(),np.nan)

for i,ss in enumerate(titles):
    tmndy=tmn_seas[ss].t2m.groupby('time.year').quantile(0.75)
    tmn_dy_seas[i,]=tmndy
tmn_dy_seas=tmn_dy_seas.to_dataset(name='htdays')

tmndy_seasch=tmn_dy_seas.chunk({'latitude': 13, 'longitude': 12,'year': -1})
slope, pval = xr.apply_ufunc(
    manntrend,
    tmndy_seasch.htdays,  # your variable (time on first axis)
    input_core_dims=[['year']],
    output_core_dims=[[], []],
    vectorize=True,
    dask='parallelized',
    output_dtypes=[float, float]
)

trseas_q75_era,pvseas_q75_era=dask.compute(slope,pval)


In [ ]:
#berkley
tmnbk_ch=tmnbk_midlat.chunk({'latitude': 26, 'longitude': 60})
tmnbk_seas=tmnbk_ch.groupby('time.season')

titles=tmnbk_seas.mean().season.values
# year= np.arange(1950,2025)
tmnbk_dy_seas=xr.full_like(tmnbk_midlat.temperature.groupby('time.year').mean().expand_dims(season=titles).copy(),np.nan)

for i,ss in enumerate(titles):
    tmnbkdy=tmnbk_seas[ss].temperature.groupby('time.year').quantile(0.75)
    tmnbk_dy_seas[i,]=tmnbkdy
tmnbk_dy_seas=tmnbk_dy_seas.to_dataset(name='htdays')

tmnbkdy_seasch=tmnbk_dy_seas.chunk({'latitude': 13, 'longitude': 12,'year': -1})
slope, pval = xr.apply_ufunc(
    manntrend,
    tmnbkdy_seasch.htdays,  # your variable (time on first axis)
    input_core_dims=[['year']],
    output_core_dims=[[], []],
    vectorize=True,
    dask='parallelized',
    output_dtypes=[float, float]
)

trseas_q75_bk,pvseas_q75_bk=dask.compute(slope,pval)

In [ ]:
#save
trseas_q75_era.to_dataset(name='htdays').to_netcdf('era5/era5_tmean_q75_trend_final_1950-2021.nc')
pvseas_q75_era.to_dataset(name='htdays').to_netcdf('era5/era5_tmean_q75_trendpvals_final_1950-2021.nc')


In [ ]:
#save
trseas_q75_bk.to_dataset(name='htdays').to_netcdf('berkeley/berkeley_tmean_q75_trend_final_1950-2021.nc')
pvseas_q75_bk.to_dataset(name='htdays').to_netcdf('berkeley/berkeley_tmean_q75_trendpvals_final_1950-2021.nc')

## Temp variability trends


### CMIP6

In [ ]:
#var seasonal mean 
temp_sdrolled=tmn_noleap21.rolling(time=10,center=True).std()
bktemp_sdrolled=tmnbk_noleap.rolling(time=10,center=True).std()
cmptemp_sdrolled=temp_mult21.rolling(time=10,center=True).std()

temp_seas=temp_sdrolled.t2m.groupby('time.season').mean()
bktemp_seas=bktemp_sdrolled.temperature.groupby('time.season').mean()
cmptemp_seas=cmptemp_sdrolled.tas.groupby('time.season').mean()

In [ ]:
##prep cmip

temp_seas=cmptemp_sdrolled.tas.groupby('time.season')
titles=temp_seas.mean().season.values
# year= np.arange(1950,2025)
temp_dy_seas=xr.full_like(cmptemp_sdrolled.tas.groupby('time.year').mean().expand_dims(season=titles).copy(),np.nan)

trseas_mn=[]
pvseas_mn=[]
for i,ss in enumerate(titles):
    tempdy=temp_seas[ss].groupby('time.year').mean()
    temp_dy_seas[i,]=tempdy
tempvar_cmip_seas=temp_dy_seas.assign_coords(models=temp_dy_seas.models).to_dataset(name='tas')

In [ ]:
#calc slope
from dask.distributed import Client
#client = Client(n_workers=20, threads_per_worker=1,  memory_limit='12GB') 
tempdy_seasch=tempvar_cmip_seas.chunk({'latitude': 13, 'longitude': 12,'year': -1})
slope, pval = xr.apply_ufunc(
    manntrend,
    tempdy_seasch.tas,  # your variable (time on first axis)
    input_core_dims=[['year']],
    output_core_dims=[[], []],
    vectorize=True,
    dask='parallelized',
    output_dtypes=[float, float]
)
trvar_mn_cmp,pvvar_mn_cmp=dask.compute(slope,pval)

In [ ]:
#save
trvar_mn_cmp.to_netcdf('cmip6_tasvar_hist_mktrends.nc')
pvvar_mn_cmp.to_netcdf('cmip6_tasvar_hist_mkpvals.nc')

### ERA5/Berkeley

In [ ]:
##prep era
temp_doysd_seas=temp_sdrolled.t2m.groupby('time.season')
titles=temp_doysd_seas.mean().season.values
# year= np.arange(1950,2025)
tmn_var_seas=xr.full_like(temp_sdrolled.t2m.groupby('time.year').mean().expand_dims(season=titles).copy(),np.nan)

for i,ss in enumerate(titles):
    tmndy=temp_doysd_seas[ss].groupby('time.year').mean()
    tmn_var_seas[i,]=tmndy
    
tmn_var_seas=tmn_var_seas.to_dataset(name='htdays')



In [ ]:
#era trend
tmndy_seasch=tmn_var_seas.chunk({'latitude': 13, 'longitude': 12,'year': -1})
slope, pval = xr.apply_ufunc(
    manntrend,
    tmndy_seasch.htdays,  # your variable (time on first axis)
    input_core_dims=[['year']],
    output_core_dims=[[], []],
    vectorize=True,
    dask='parallelized',
    output_dtypes=[float, float]
)

trvar_mn_era,pvvar_mn_era=dask.compute(slope,pval)

In [ ]:
#prep berkley
bktemp_doysd_seas=bktemp_sdrolled.temperature.groupby('time.season')
titles=bktemp_doysd_seas.mean().season.values
# year= np.arange(1950,2025)
bktmn_var_seas=xr.full_like(bktemp_sdrolled.temperature.groupby('time.year').mean().expand_dims(season=titles).copy(),np.nan)

for i,ss in enumerate(titles):
    bktmndy=bktemp_doysd_seas[ss].groupby('time.year').mean()
    bktmn_var_seas[i,]=bktmndy
    
bktmn_var_seas=bktmn_var_seas.to_dataset(name='htdays')




In [ ]:
#berkley trend
bktmndy_seasch=bktmn_var_seas.chunk({'latitude': 13, 'longitude': 12,'year': -1})
slope, pval = xr.apply_ufunc(
    manntrend,
    bktmndy_seasch.htdays,  # your variable (time on first axis)
    input_core_dims=[['year']],
    output_core_dims=[[], []],
    vectorize=True,
    dask='parallelized',
    output_dtypes=[float, float]
)

trvar_mn_bk,pvvar_mn_bk=dask.compute(slope,pval)

In [ ]:
#save
trvar_mn_era.to_netcdf('era5/era5_tmeanvar_trend_final_1950-2021.nc')
pvvar_mn_era.to_netcdf('era5/era5_tmeanvar_trendpvals_final_1950-2021.nc')


In [ ]:
#save
trvar_mn_bk.to_netcdf('berkeley/berkeley_tmeanvar_trend_final_1950-2021.nc')
pvvar_mn_bk.to_netcdf('berkeley/berkeley_tmeanvar_trendpvals_final_1950-2021.nc')

# Future trends - CMIP6

### Hotspells

In [ ]:
#models
###'HadGEM3-GC31-MM'
models1=['ACCESS-CM2','AWI-CM-1-1-MR','BCC-CSM2-MR', 'CanESM5','CESM2','CMCC-ESM2','CNRM-ESM2-1','EC-Earth3-CC','GFDL-ESM4','IITM-ESM','INM-CM5-0','KIOST-ESM','MIROC6','MPI-ESM1-2-LR','MRI-ESM2-0','NESM3','NorESM2-MM','TaiESM1']# temp_datcut={}
models22=['CESM2-WACCM', 'CMCC-CM2-SR5', 'CNRM-CM6-1','INM-CM4-8','IPSL-CM6A-LR', 'MIROC-ES2L','CNRM-CM6-1-HR']
models=models1+models22#['ACCESS-CM2','AWI-CM-1-1-MR','BCC-CSM2-MR', 'CanESM5','CESM2', 'CESM2-WACCM','CMCC-ESM2', 'CMCC-CM2-SR5','CNRM-ESM2-1', 'CNRM-CM6-1', 'CNRM-CM6-1-HR','EC-Earth3-CC','GFDL-ESM4','IITM-ESM','INM-CM4-8', 'INM-CM5-0', 'IPSL-CM6A-LR','KIOST-ESM','MIROC6', 'MIROC-ES2L','MPI-ESM1-2-LR','MRI-ESM2-0','NESM3','NorESM2-MM','TaiESM1']
len(models)

In [ ]:
#get hw data
import dask
heat_datfut={}
toconc_fut=[]
for i,mod in enumerate(models):
    heat=xr.open_dataset('cmip6/heat/'+mod+'_tmean_heatdays_future_ssp5_2025_2100_regridcon.nc')#.rename({'lat':'latitude','lon':'longitude'})
    heat1=heat.sortby(heat.time)
    heat_midl=cutmidlat(heat1)
    heat_midlat=landmask(heat_midl).sel(time=slice('2030-01-01','2099-12-31'))
    if i >0:
        heat_midlat['time']=heat_datfut['ACCESS-CM2'].time.values
    else:
        pass
    heat_datfut[mod]=heat_midlat
    toconc_fut.append(heat_midlat)

In [ ]:
#concatenate models
heat_multfut=xr.concat(toconc_fut,dim='models')


In [ ]:
#save
# heat_multfut.to_netcdf('cmip5/heat/cmip6_tas_heat_future.nc')


In [ ]:
#re read
heat_multfut70=heat_multfut.sel(time=slice('2030-01-01','2099-12-31'))

heat_multhist70=heat_mult.sel(time=slice('1950-01-01','2019-12-31'))

In [ ]:
#seasonal
heat_futseas=heat_multfut70.groupby('time.season').mean()
heat_histseas=heat_multhist70.groupby('time.season').mean()

heat_difseas=heat_futseas-heat_histseas

In [ ]:
##prepare data
heat_seasfuture=heat_multfut70.groupby('time.season')
titles=heat_seasfuture.mean().season.values
# year= np.arange(1950,2025)
heat_dy_seasfut=xr.full_like(heat_multfut70.htdays.groupby('time.year').mean().expand_dims(season=titles).copy(),np.nan)

trseas_mn=[]
pvseas_mn=[]
for i,ss in enumerate(titles):
    heatdy=heat_seasfuture[ss].htdays.groupby('time.year').mean()
    heat_dy_seasfut[i,]=heatdy
heat_dy_seasfut=heat_dy_seasfut.assign_coords(models=heat_dy_seasfut.models).to_dataset(name='htdays')

In [ ]:
#calc slope
from dask.distributed import Client
client = Client(n_workers=20, threads_per_worker=1,  memory_limit='12GB') 
heatdy_seasch=heat_dy_seasfut.chunk({'latitude': 13, 'longitude': 12,'year': -1})
slope, pval = xr.apply_ufunc(
    manntrend,
    heatdy_seasch.htdays,  # your variable (time on first axis)
    input_core_dims=[['year']],
    output_core_dims=[[], []],
    vectorize=True,
    dask='parallelized',
    output_dtypes=[float, float]
)

#compute dask
trseas_heatfut,pvseas_heatfut=dask.compute(slope,pval)

In [ ]:
trseas_heatfut.to_dataset(name='tas').to_netcdf('cmip6_heat_fut_mktrends.nc')
pvseas_heatfut.to_dataset(name='tas').to_netcdf('cmip6_heat_fut_mkpvals.nc')

### Temperature

In [ ]:
#temperature
to_con=[]
# temp_datcut={}
for mod in models1:
    dat=xr.open_dataset('cmip6/future/tas_day_'+mod+'_future_ssp5_regridcon.nc')#.rename({'lat':'latitude','lon':'longitude'})
    dat1=dat.convert_calendar('noleap').rename({'lon':'longitude','lat':'latitude'})
    temp_dat=rotlon_180(dat1)
    tmp1=temp_dat.sortby(temp_dat.time)
    tmp_midl=cutmidlat(tmp1)
    tmp_midlat=landmask(tmp_midl)
    tmp_midlat=tmp_midlat.drop_duplicates(dim="time")
   # temp_datcut[mod]=temp_dat#.sel(time=(slice('1950-01-01','2024-12-31')))
    to_con.append(tmp_midlat)

models2=['cesm2_waccm', 'cmcc_cm2_sr5', 'cnrm_cm6_1','inm_cm4_8','ipsl_cm6a_lr', 'miroc_es2l','cnrm_cm6_1_hr']
# temp_datcut={}
for mod in models2:
    dat=xr.open_dataset('cmip6/tmean_ssp/tmean_'+mod+'_sspfuture_regridcon.nc')#.rename({'lat':'latitude','lon':'longitude'})
    dat1=dat.convert_calendar('noleap').rename({'lon':'longitude','lat':'latitude'})
    temp_dat=rotlon_180(dat1)
    tmp1=temp_dat.sortby(temp_dat.time)
    tmp_midl=cutmidlat(tmp1)
    tmp_midlat=landmask(tmp_midl)
    tmp_midlat=tmp_midlat.drop_duplicates(dim="time")
   # temp_datcut[mod]=temp_dat#.sel(time=(slice('1950-01-01','2024-12-31')))
    to_con.append(tmp_midlat) 

#temp_multfut=xr.open_dataset('cmip6/future/tas_day_allmodels_future_ssp5_regridcon.nc')

In [ ]:
#concatenate
temp_multfut=xr.concat(to_con,dim='models')
temp_multfut70=temp_multfut.sel(time=slice('2030-01-01','2099-12-31'))

temp_multhist70=temp_mult.sel(time=slice('1950-01-01','2019-12-31'))

In [ ]:
##mann kendall version

temp_seas=temp_multfut70.tas.groupby('time.season')
titles=temp_seas.mean().season.values
# year= np.arange(1950,2025)
temp_dy_seas=xr.full_like(temp_multfut70.tas.groupby('time.year').mean().expand_dims(season=titles).copy(),np.nan)

trseas_mn=[]
pvseas_mn=[]
for i,ss in enumerate(titles):
    tempdy=temp_seas[ss].groupby('time.year').mean()
    temp_dy_seas[i,]=tempdy
temp_dy_seas=temp_dy_seas.assign_coords(models=temp_dy_seas.models).to_dataset(name='tas')

In [ ]:
#calc slope
from dask.distributed import Client
#client = Client(n_workers=20, threads_per_worker=1,  memory_limit='12GB') 
tempdy_seasch=temp_dy_seas.chunk({'latitude': 13, 'longitude': 12,'year': -1})
slope, pval = xr.apply_ufunc(
    manntrend,
    tempdy_seasch.tas,  # your variable (time on first axis)
    input_core_dims=[['year']],
    output_core_dims=[[], []],
    vectorize=True,
    dask='parallelized',
    output_dtypes=[float, float]
)
trseas_tmpfut,pvseas_tmpfut=dask.compute(slope,pval)

In [ ]:
#save
trseas_tmpfut.to_dataset(name='tas').to_netcdf('cmip6_tmean_trend_final_2030-2099.nc')
pvseas_tmpfut.to_dataset(name='tas').to_netcdf('cmip6_tmean_trendpvals_final_2030-2099.nc')